## Creating a cartesian mesh with sinusoidal boundary

Author: W Eaton (weaton@princeton.edu) \
Last updated: 23/05/2022


### Prerequisites/dependencies

We will need to have the following installed: 
* SPECFEM2D  - [available here on Github](https://github.com/SPECFEM/specfem2d)
* MeshAssist - [available here on Github](https://github.com/homnath/MeshAssist)
* Cubit      - You can get a free academic (trial license) from [their website](https://coreform.com/products/coreform-cubit/free-meshing-software/)

The Cubit trial version allows you to generate small-to-medium sized meshes. For 3D it is limited to 50,000 hexahedra. I'm not sure what it is for 2D. Of course, each of these packages have a number of their own dependencies. See their own respective manuals for installation. 


### Some Cubit basics

Cubit has a GUI for interactive mesh building, but is fundamentally driven by python code. There are (annoyingly) some minor differences between Cubit Python and normal Python - there are some notes on this in the [Mesh Generation](https://specfem2d.readthedocs.io/en/latest/03_mesh_generation/) section of the manual, but it wont affect us much. 


Cubit has an object-oriented structure. Objects include: 

* Vertices - individual points 
* Curves   - made up of points
* Surfaces - defined by bounding curves
* Volumes  - defined by bounding surfaces

For using SPECFEM2D we will only need to worry about vertices, curves, and surfaces.  


When you first open CUBIT, the screen will look something like this: 
![blank_cubit](./notebook_figs/cubit_blank.png)


The left Panel 'Model Tree' shows you a tree-structure of the different objects you have created. Below this the 'Properties Page' lists properties of a selected object - more on this later. On the right, the 'Command Panel' is the main GUI for creating and modifying objects. Finally, the bottom terminal allows you to enter Python/Cubit commands. The history tab is also useful for keeping a copy of all the command you have previously entered. 


For creating objects you have two options: 
* Use the GUI Command Panel
* Enter commands directly


For experimenting, the GUI is very useful - but you dont want to have to use it for creating 100 vertices individiually! In this case, it is easier to generate commands somehow, and then copy-paste them into the terminal. This is what we will do below. 


### Making our mesh: 
Lets start by defining some dimensions. We will create a box with an internal, sinusoidal surface. 

In [4]:
import numpy as np
import matplotlib.pyplot as plt


# Define dimensions: e.g. from 0 to 28 km in x direction and 0 to 10 in y direction
x_min = 0
x_max = 9*np.pi
y_min = 0
y_max = -10
nx    = 100 # Number of points in x
ny    = 50  # Number of points in y

# Define x,y arrays:
x = np.linspace(x_min, x_max, nx)
y = np.linspace(y_min, y_max, ny)

Now we are going to print a number of commands that will generate vertices. These vertices lie on the sinusoid. The command has the following syntax: 

```create vertex  <x-coordinate>  <y-coordinate>  <z-coordinate>```

We are only doing 2D so keep the z coordinate at a constant value (e.g. zero). If we want to specify the colour of the object we can use 

```create vertex  <x-coordinate>  <y-coordinate>  <z-coordinate>  colour  <colour>```. 

Run the box below and copy all of the text that is printed. Paste the commands into the python terminal in Cubit. You should now see a number of vertices that look lie along a sinusoid.

In [6]:
# print top side (sinusoid)
sin_array = np.sin(x)
for i in range(nx):
    print(f"create vertex {x[i]} {sin_array[i]} 0 color brown")

create vertex 0.0 0.0 0 color brown
create vertex 0.28559933214452665 0.28173255684142967 0 color brown
create vertex 0.5711986642890533 0.5406408174555976 0 color brown
create vertex 0.8567979964335799 0.7557495743542582 0 color brown
create vertex 1.1423973285781066 0.9096319953545183 0 color brown
create vertex 1.4279966607226333 0.9898214418809327 0 color brown
create vertex 1.7135959928671598 0.9898214418809328 0 color brown
create vertex 1.9991953250116865 0.9096319953545184 0 color brown
create vertex 2.284794657156213 0.7557495743542584 0 color brown
create vertex 2.5703939893007397 0.5406408174555978 0 color brown
create vertex 2.8559933214452666 0.2817325568414296 0 color brown
create vertex 3.141592653589793 1.2246467991473532e-16 0 color brown
create vertex 3.4271919857343196 -0.28173255684142945 0 color brown
create vertex 3.7127913178788465 -0.5406408174555976 0 color brown
create vertex 3.998390650023373 -0.7557495743542582 0 color brown
create vertex 4.2839899821679 -0.

We now need the bottom vertices that will define the bottom of the mesh. To get some practice using the GUI, go to the right-hand-side panel called the *Command Panel*. Click on the BLUE icon called *Geometry*. Click on the button called *Create Geometry*. Here you will see that we can create volumes, surfaces, curves, or vertices. Click on *Create vertices*. 

A vertex can be defined in a number of ways by choosing an option from the drop-down bar. By default *Arc Centre* is selected. I find the easiest method is *location*. Select the *location* option.

In the *Location...* box, write the coordinates:  28.274333882308138 -10 0.

You can pick a colour if you like, then click Apply. Now do the same with the coordinates 0 -10 0 

Now we have all the vertices needed to make our first surface!


### Making our first surface

Now that we have all our vertices, we need to combine them. There is a very subtle, and important thing to know here about cubit surfaces. Since a surface is defined by a normal, there are two possible directions that this normal could point in. Cubit has a convention for numbering nodes on a surface. Viewing the surface from the outside, the corner nodes are numbered in an anti-clockwise direction, as shown in the figure below for nodes 1, 2, 3, 4: 
![blank_cubit](./notebook_figs/node_orders.png) **Because we are only using a surface, it doesnt really matter which side is 'outward'. That means it doesnt matter if you define your nodes on a surface clockwise, or anticlockwise, but you MUST be consistent. Here we will define the clockwise**. 

When creating a surface, we can build it from nodes, curves, or a number of other ways. Here we will stick with using the vertices we defined. On the left-hand panel you should see a list of the 'Free Vertices' - i.e. vertices that are not currently defining another object like a curve/surface. 

![blank_cubit](./notebook_figs/vertex_list.png)


To define a surface using our nodes we need to use the command 

``` create surface vertex <list of vertices in order>``` 

This will build a surface using sucessive curves between the vertices. If the list of vertices is not in the correct order then the surface will be wrong. Overall we have 102 vertices. This is a good example of where the GUI is not a good idea because you dont want to type out 102 numbers successively. Instead, lets generate the command by running the cell below: 

In [8]:
s = "create surface vertex "
for i in range(102):
    s += f"{i+1} "
print(s)

create surface vertex 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 


You should now have your first surface! To emphasise how important this ordering is, you can try and break the program. If you click the UNDO button at the top (blue arrow) you should recover your set of vertices. Try running the above command but switching the order of two of the node IDs...e.g. swap 78 and 100 - you should now see a very bad surface!  

### Making our second surface

This time we will do a similar thing. Run the following cell and get the commands. You will notice this time I have defined the sinusoidal nodes in reverse order so that they are successively being created in a clockwise direction.

In [13]:
print(f"create vertex {x_min} 4 0 color brown")
print(f"create vertex {x_max} 4 0 color brown")
for i in range(1,nx):
    print(f"create vertex {x[-i]} {sin_array[-i]} 0 color brown")

print(f"create vertex {x[0]} {sin_array[0]} 0 color brown")

create vertex 0 4 0 color brown
create vertex 28.274333882308138 4 0 color brown
create vertex 28.274333882308138 1.102182119232618e-15 0 color brown
create vertex 27.988734550163613 0.2817325568414289 0 color brown
create vertex 27.703135218019085 0.5406408174555982 0 color brown
create vertex 27.417535885874557 0.7557495743542598 0 color brown
create vertex 27.131936553730032 0.9096319953545186 0 color brown
create vertex 26.846337221585504 0.989821441880933 0 color brown
create vertex 26.56073788944098 0.9898214418809327 0 color brown
create vertex 26.27513855729645 0.9096319953545177 0 color brown
create vertex 25.989539225151926 0.7557495743542585 0 color brown
create vertex 25.703939893007398 0.5406408174555964 0 color brown
create vertex 25.418340560862873 0.2817325568414303 0 color brown
create vertex 25.132741228718345 -9.797174393178826e-16 0 color brown
create vertex 24.84714189657382 -0.2817325568414288 0 color brown
create vertex 24.561542564429292 -0.5406408174555981 0 co

Now create the associated surface by copying the commands generated below: 

In [14]:
s = "create surface vertex "
for i in range(102,204):
    s += f"{i+1} "
print(s)

create surface vertex 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 


## Meshing our domain

We are now ready to mesh our domain. We have 2 surfaces that we want to combine in a single mesh. Note that if you mesh these surfaces separately, there is no guarantee the mesh will line up along the boundary of this mesh, and that will cause SPECFEM to break. 

From the *Command Panel*, select the purple icon, then click *Surface* (the second option). Click on 'Define intervals and sizes'. From the drop-down, select *Approximate Size*. In the box that says Select Surfaces, enter the ID's of our two surface (1 and 2). 

Enter an *Approximate Size* value of 0.3. Click *Apply Size* and then *Mesh*. Now we have a mesh! If you choose a smaller size, you will have a more detailed mesh, but it will have more elements - this means slower computation. If there are too many, Cubit may also not allow you to export the mesh using a free license! 


You can also do this meshing using the following commands: 

```
surface 1 2 size 0.3
surface 1 2 size 0.3
mesh surface 1 2
```

### Material parameters and boundaries


SPECFEM will need to know 
1. Which edges are free surfaces/absorbing boundaries
2. Which parts of the mesh are made of each material


To accomplish this, we need to use Blocks and Sidesets. 

#### Material parameters 

From the *Command Panel*, click on the Orange button for *Analysis Groups and materials*. Then click on the 3rd option called *Blocks*. 

 Each *Block* can contain a set of surfaces/volumes/curves etc. In our case, it each block will represent a different material. 
 
 Click on the first option *Create material blocks and beams* and select *Create block* from the dropdown. You do not need to add in a *Block ID*, but you should add a name: e.g. 'Sandstone' or 'Material 1'. 
 
 Click on the *surface* option to make a block that contains various surfaces, and add in the ID of the surface you desire. In our example, we have only 2 surfaces and they will each be part of a different block, since we want them to be different materials - so just add in the number 1, for the ID of the first element. Now click *Apply*. 
 
 On the Left-Hand-Side panel (*Model Tree*) you should now see that a Block has been created. 
 
 We could have also made this block by entering the command 
 
 
 ``` block 1 add surface 1 ```
 
 where the syntax is 
 
 ``` block <block_ID> add surface <surface_ID>```. 
 
 To practice this, make the second block (material) by entering the command 
 
  ``` block 2 add surface 2 ```. 
  
  
Each of these blocks needs to have elements in the format QUAD4. To make sure this is true, enter the commands: 

```
block 1 element type QUAD4
block 2 element type QUAD4
```


#### Boundary Conditions

The top surface of our mesh will be a free surface, and the side edges will be absorbing boundaries. You do not need to worry about internal boundaries, where boundary conditions such as continuity of traction and displacement etc will be applied by SPECFEM. 

To indicate which boundaries/curves should have a free-surface boundary condition, we will make a sideset. 

In the *Command Panel* click the Orange Icon again, and this time select *Sidesets* instead of *Blocks* as before. Here you can create a sideset using the same method as blocks. To be quicker, we will use the commands: 

```
sideset 1 add curve 103  
sideset 1 name "free_surface"


sideset 2 add curve 104 100 101 102 204 
sideset 2 name "absorbing_boundary"
```

Above, the numbers 104 etc are the IDs of the curves that make up the boundary. 


**AND WE ARE FINISHED!** Our mesh is ready to be exported. 


## Using our mesh in SPECFEM2D


We now need to convert our mesh to something SPECFEM2D can read. This is where MeshAssist can help. The steps are as follows: 

1. Export the mesh from CUBIT 
2. Run MeshAssist
3. Move/Copy the output files to SPECFEM2D 


### 1. Export the mesh from CUBIT 

Once you are happy with your mesh, go to File > Export

Export your mesh as a ```.e``` file to your desired location. A box will appear with 2 tickboxes. Tick the box called *Advanced*, and select the following: 
* Dimension: 3d (yes I know it is a 2d mesh but click 3d!)
* Format: Use Large File Format

Then click finish. Your mesh will now be exported. You will need a copy of this file in the ```bin``` directory of MeshAssist, so move or copy it there.  


### 2. Run MeshAssist

Once your Exodus (.e) file is in the the ```bin``` directory of MeshAssist you can run the binary file ```exodus2specfem2d``` with the command:

```./exodus2specfem2d sin.e -bin=1 -fac=1000 -head=1 -order=1```

All of the optional arguments are explained in the Manual. Note that the fac=1000 multiplies our measurements by 1000 so that the box is now 28 km, rather than 28 metres, wide. 

If MeshAssist runs successfully it will output a number of files. We will need the following files: 

* Connectivity
* Coordinates
* Free Surface
* Absorbing Boundary
* Materials

Move these files to the SPECFEM2D directory. Specifically I would store them in ```SPECFEM2D/DATA/sinusoid_example```. You will need to make this subdirectory in the ```DATA``` directory. 


**We are now ready to run our simulation**